In [3]:
import sys
import os
import hashlib
import struct
import subprocess
import collections
import tensorflow as tf
from tensorflow.core.example import example_pb2
#chunk file

finished_files_dir = "finished_files"
chunks_dir = os.path.join(finished_files_dir, "chunked3")
CHUNK_SIZE = 1000  # num examples per chunk, for the chunked data

def chunk_file(set_name):
    in_file = 'finished_files/%s.bin' % set_name
    reader = open(in_file, "rb")
    chunk = 0
    finished = False
    while not finished:
        chunk_fname = os.path.join(chunks_dir, '%s_%03d.bin' % (set_name, chunk))  # new chunk
        with open(chunk_fname, 'wb') as writer:
            for _ in range(CHUNK_SIZE):
                len_bytes = reader.read(8)
                if not len_bytes:
                    finished = True
                    break
                str_len = struct.unpack('q', len_bytes)[0]
                example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
                writer.write(struct.pack('q', str_len))
                writer.write(struct.pack('%ds' % str_len, example_str))
            chunk += 1


def chunk_all():
    # Make a dir to hold the chunks
    if not os.path.isdir(chunks_dir):
        os.mkdir(chunks_dir)
    # Chunk the data
    for set_name in ['train', 'val']:
        print("Splitting %s data into chunks..." % set_name)
        chunk_file(set_name)
    print("Saved chunked data in %s" % chunks_dir)
chunk_all()

Splitting train data into chunks...
Splitting val data into chunks...
Saved chunked data in finished_files/chunked3


In [ ]:
import sys
import os
import hashlib
import struct
import subprocess
import collections
import tensorflow as tf
from tensorflow.core.example import example_pb2
import json

In [ ]:
#write train_data

for i in range(0,9):
    print(i)
    file_object = open(r"bytecup2018/bytecup.corpus.train."+str(i)+".txt")
    try:
        all_the_text = file_object.readlines()
        print(len(all_the_text))
        #print(all_the_text[0])
        #print(all_the_text[1])
        #print(json.loads(all_the_text))


        for text in all_the_text:
            arr = json.loads(text)
            #print(len(arr))
            train_id = arr['id']
            if train_id%10000==0:
                print(train_id)
            # Write to file
            train_file = os.path.join('bytecup2018/train_data', "%0d.txt" % int(train_id))


            with open(train_file, "w") as f:
                f.write(arr['content'] + "\n\n")
                f.write('@highlight' + "\n\n")
                f.write(arr['title'])

        #for a in arr:
         #    print(a)
    finally:
        file_object.close()

In [ ]:
#write test_data

file_object = open(r"bytecup2018/bytecup.corpus.test_set.txt")
try:
    all_the_text = file_object.readlines()
    print(len(all_the_text))
    #print(all_the_text[0])
    #print(all_the_text[1])
    #print(json.loads(all_the_text))


    for text in all_the_text:
        arr = json.loads(text)
        #print(len(arr))
        train_id = arr['id']

        # Write to file
        train_file = os.path.join('bytecup2018/test_data', "%0d.txt" % int(train_id))


        with open(train_file, "w") as f:
            f.write(arr['content'])
            #f.write('@highlight' + "\n\n")
            #f.write(arr['title'])

    #for a in arr:
     #    print(a)
finally:
    file_object.close()

In [ ]:
#Stanford CoreNLP Tokenizer

stories_dir = "bytecup2018/test_data"
tokenized_stories_dir = "bytecup2018/tokenized_test_data"

"""Maps a whole directory of .story files to a tokenized version using Stanford CoreNLP Tokenizer"""
print("Preparing to tokenize %s to %s..." % (stories_dir, tokenized_stories_dir))
stories = os.listdir(stories_dir)
# make IO list file
print("Making list of files to tokenize...")
with open("mapping.txt", "w") as f:
    for s in stories:
        f.write("%s \t %s\n" % (os.path.join(stories_dir, s), os.path.join(tokenized_stories_dir, s)))
command = ['java', 'edu.stanford.nlp.process.PTBTokenizer', '-ioFileList', '-preserveLines', 'mapping.txt']
print("Tokenizing %i files in %s and saving in %s..." % (len(stories), stories_dir, tokenized_stories_dir))
subprocess.call(command)
print("Stanford CoreNLP Tokenizer has finished.")
os.remove("mapping.txt")

# Check that the tokenized stories directory contains the same number of files as the original directory
num_orig = len(os.listdir(stories_dir))
num_tokenized = len(os.listdir(tokenized_stories_dir))
if num_orig != num_tokenized:
    raise Exception(
        "The tokenized stories directory %s contains %i files, but it should contain the same number as %s (which has %i files). Was there an error during tokenization?" % (
        tokenized_stories_dir, num_tokenized, stories_dir, num_orig))
print("Successfully finished tokenizing %s to %s.\n" % (stories_dir, tokenized_stories_dir))

In [ ]:
#lea1_2

decode_dir ='bytecup2018/test_data'
for file in os.listdir(decode_dir):
    file_name = decode_dir + "/" +file
    with open(file_name,"r") as f:
        decode = f.read()
        #print(decode)
        with open("submit/test_lead1_2/"+file,"w") as wf:
            print(file)
            if len(decode.split(".")[0])>3:
                wf.write(decode.split(".")[0])
            elif len(decode)<3:
                wf.write(decode)
            else:
                wf.write(decode.split(".")[0]+"."+decode.split(".")[1])

In [ ]:
#definition write to bin function

def read_text_file(text_file):
    lines = []
    with open(text_file, "r") as f:
        for line in f:
            lines.append(line.strip())
    return lines

dm_single_close_quote = u'\u2019'  # unicode
dm_double_close_quote = u'\u201d'
END_TOKENS = ['.', '!', '?', '...', "'", "`", '"', dm_single_close_quote, dm_double_close_quote,
              ")"]  # acceptable ways to end a sentence

# We use these to separate the summary sentences in the .bin datafiles
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'

def fix_missing_period(line):
    """Adds a period to a line that is missing a period"""
    if "@highlight" in line: return line
    if line == "": return line
    if line[-1] in END_TOKENS: return line
    # print line[-1]
    return line + " ."

def get_art_abs(story_file):
    lines = read_text_file(story_file)

    # Lowercase everything
    lines = [line.lower() for line in lines]

    # Put periods on the ends of lines that are missing them (this is a problem in the dataset because many image captions don't end in periods; consequently they end up in the body of the article as run-on sentences)
    lines = [fix_missing_period(line) for line in lines]

    # Separate out article and abstract sentences
    article_lines = []
    highlights = []
    next_is_highlight = False
    for idx, line in enumerate(lines):
        if line == "":
            continue  # empty line
        elif line.startswith("@highlight"):
            next_is_highlight = True
        elif next_is_highlight:
            highlights.append(line)
        else:
            article_lines.append(line)

    # Make article into a single string
    article = ' '.join(article_lines)

    # Make abstract into a signle string, putting <s> and </s> tags around the sentences
    abstract = ' '.join(["%s %s %s" % (SENTENCE_START, sent, SENTENCE_END) for sent in highlights])

    return article, abstract

In [ ]:
#write to bin

root_dir = 'bytecup2018/tokenized_test_data'
out_file = 'finished_files/test.bin'
i = 0

with open(out_file, 'ab') as writer:
    for story_file in os.listdir(root_dir):
        i = i+1
        if i%10000==0:
            print(i)
        file = os.path.join("bytecup2018/train_data",story_file)
        article, abstract = get_art_abs(file)
        #print(story_file)
        #print(article)
        #print("-------------")
        #print(abstract)
        # Write to tf.Example
        tf_example = example_pb2.Example()
        tf_example.features.feature['article'].bytes_list.value.extend([article.encode()])
        tf_example.features.feature['abstract'].bytes_list.value.extend([abstract.encode()])
        tf_example_str = tf_example.SerializeToString()
        str_len = len(tf_example_str)
        writer.write(struct.pack('q', str_len))
        writer.write(struct.pack('%ds' % str_len, tf_example_str))
print("Finished writing file %s\n" % out_file)

In [ ]:
#chunk file

finished_files_dir = "finished_files"
chunks_dir = os.path.join(finished_files_dir, "chunked3")
CHUNK_SIZE = 1000  # num examples per chunk, for the chunked data

def chunk_file(set_name):
    in_file = 'finished_files/%s.bin' % set_name
    reader = open(in_file, "rb")
    chunk = 0
    finished = False
    while not finished:
        chunk_fname = os.path.join(chunks_dir, '%s_%03d.bin' % (set_name, chunk))  # new chunk
        with open(chunk_fname, 'wb') as writer:
            for _ in range(CHUNK_SIZE):
                len_bytes = reader.read(8)
                if not len_bytes:
                    finished = True
                    break
                str_len = struct.unpack('q', len_bytes)[0]
                example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
                writer.write(struct.pack('q', str_len))
                writer.write(struct.pack('%ds' % str_len, example_str))
            chunk += 1


def chunk_all():
    # Make a dir to hold the chunks
    if not os.path.isdir(chunks_dir):
        os.mkdir(chunks_dir)
    # Chunk the data
    for set_name in ['train', 'val']:
        print("Splitting %s data into chunks..." % set_name)
        chunk_file(set_name)
    print("Saved chunked data in %s" % chunks_dir)
chunk_all()

In [ ]:
#drop unk

# root_dir为要读取文件的根目录
decode_dir = "submit/decoded"
output_dir = "submit/result"
# 读取批量文件后要写入的文件
#with open("abstract.txt", "w") as abstract:

# 依次读取根目录下的每一个文件
for file in os.listdir(decode_dir):
    file_name = decode_dir + "/" + file
    with open(file_name, 'r') as f:
        sentence=f.read()
        with open(output_dir + "/" + file, 'w') as wf:
            s = sentence.replace("[UNK] ","")
            wf.write(s)


In [ ]:
#修改decode檔名

# -*- coding:utf-8 -*-
import os

# root_dir为要读取文件的根目录
decode_dir = "submit/result"
# 读取批量文件后要写入的文件
#with open("abstract.txt", "w") as abstract:

# 依次读取根目录下的每一个文件
for file in os.listdir(decode_dir):
    file_name = decode_dir + "/" + file
    file = str(int(file[:6])+1)
    refer_name = decode_dir +"/"+file+".txt"
    os.rename(file_name, refer_name)